In [21]:
import importlib
import os
import fnmatch
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import csv
from sklearn import tree
import sys
sys.stdout.flush()
import math
import matplotlib.pyplot as plt

In [80]:
clean_belloni = open('1915Belloniclass_updated.dat')
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state

available = []
pool=[]

#/home/jkok1g14/Documents/GRS1915+105/data
#/export/data/jakubok/GRS1915+105/Std1_PCU2
for root, dirnames, filenames in os.walk("/home/jkok1g14/Documents/GRS1915+105/data/Std1_PCU2"):
    for filename in fnmatch.filter(filenames, "*_std1_lc.txt"):
        available.append(filename)
for ob, state in ob_state.items():
    if ob+"_std1_lc.txt" in available:
        pool.append(ob)  

#create a list of arrays with time and counts for the set of Belloni classified observations
lc_dirs=[]
lcs=[]
ids=[]
for root, dirnames, filenames in os.walk("/home/jkok1g14/Documents/GRS1915+105/data/Std1_PCU2"):    
    for filename in fnmatch.filter(filenames, "*_std1_lc.txt"):
        if filename.split("_")[0] in pool:
            lc_dirs.append(os.path.join(root, filename))

            
#make 2D arrays for light curves, with columns of counts and time values
for lc in lc_dirs:
    ids.append(lc.split("/")[-1].split("_")[0])
    f=np.loadtxt(lc)
    f=np.transpose(f)#,axis=1)
    f=f[0:2]
    ###1s average and time check to eliminate points outside of GTIs
    f8t = np.mean(f[0][:(len(f[0])//8)*8].reshape(-1, 8), axis=1)
    f8c = np.mean(f[1][:(len(f[1])//8)*8].reshape(-1, 8), axis=1)
    f8c=f8c-np.mean(f8c)#normalisation/mean centering/whatever you desire most
    rm_points = []
    skip=False
    for i in range(len(f8t)-1):
        if skip==True:
            skip=False
            continue
        delta = f8t[i+1]-f8t[i]
        if delta > 1.0:
            rm_points.append(i+1)
            skip=True
            
####### normalise the count rates! think about the effect of 0-1 normalisation on the distance calculation
            
    times=np.delete(f8t,rm_points)
    counts=np.delete(f8c,rm_points)
    lcs.append(np.stack((times,counts)))
    
lc_classes=[]
for i in ids:
    lc_classes.append(ob_state[i])
lc_classes

drop_classes=[]
for clas, no in Counter(lc_classes).items():
    if no<7:
        drop_classes.append(clas)

lcs_abu = []
classes_abu = []
ids_abu = []
for n, lc in enumerate(lc_classes):
    if lc not in drop_classes:
        classes_abu.append(lc)
        lcs_abu.append(lcs[n])
        ids_abu.append(ids[n])  
x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs_abu, classes_abu, ids_abu, test_size=0.5, random_state=0, stratify=classes_abu)

In [81]:
seg_len=50
seg_slide=5
ts=x_train[0]
segments=[]
for start in range(0, len(ts[0])-seg_len, seg_slide):
    end=start+seg_len
    if ts[0][end]-ts[0][start] != seg_len:
        continue
    segments.append(np.copy(ts[1][start:end]))

In [83]:
window_rads = np.linspace(0, np.pi, seg_len)
window = np.sin(window_rads)**2
w_segments=[]
for segment in segments:
    w_segments.append(segment*window)

In [101]:
from sklearn.cluster import KMeans
cluster=KMeans(n_clusters=100, random_state=0).fit(w_segments)
centroids=cluster.cluster_centers_

In [103]:
seg_len=50
seg_slide=25
ts=x_train[0]
test_segments=[]
for start in range(0, len(ts[0])-seg_len, seg_slide):
    end=start+seg_len
    if ts[0][end]-ts[0][start] != seg_len:
        continue
    test_segments.append(np.copy(ts[1][start:end])*window)

In [112]:
prprpr=cluster.predict(np.array(test_segments[0]).reshape(1, -1))
#plt.plot(prprpr)
prprpr

array([9], dtype=int32)

In [98]:
id_train[0]

'10408-01-10-00'

In [100]:
np.where(np.array(id_train)=="10408-01-21-00")

(array([49]),)

In [99]:
ob_state

{'20187-02-01-00': 'alpha',
 '20187-02-01-01': 'alpha',
 '20402-01-22-00': 'alpha',
 '20402-01-23-00': 'alpha',
 '20402-01-24-01': 'alpha',
 '20402-01-27-00': 'alpha',
 '20402-01-28-00': 'alpha',
 '20402-01-30-02': 'alpha',
 '10408-01-10-00': 'beta',
 '10408-01-21-00': 'beta',
 '20402-01-43-02': 'beta',
 '20402-01-44-00': 'beta',
 '20402-01-45-03': 'beta',
 '20402-01-46-00': 'beta',
 '20402-01-52-01': 'beta',
 '20402-01-52-02': 'beta',
 '20402-01-53-00': 'beta',
 '40703-01-18-00': 'beta',
 '40703-01-19-00': 'beta',
 '40703-01-19-01': 'beta',
 '40703-01-22-00': 'beta',
 '40703-01-22-01': 'beta',
 '40703-01-35-00': 'beta',
 '40703-01-35-01': 'beta',
 '20402-01-37-00': 'gamma',
 '20402-01-37-02': 'gamma',
 '20402-01-38-00': 'gamma',
 '20402-01-39-00': 'gamma',
 '20402-01-39-02': 'gamma',
 '20402-01-40-00': 'gamma',
 '20402-01-55-00': 'gamma',
 '20402-01-56-00': 'gamma',
 '20402-01-57-00': 'gamma',
 '40703-01-13-00': 'gamma',
 '40115-01-07-00': 'gamma',
 '40703-01-31-00': 'gamma',
 '10408-